# Cleaned Version of Online Sales Data

The [dataset](https://www.kaggle.com/datasets/arnavsmayan/online-retail-sales-dataset) will provide synthetic data for online retail sales, which include details of customer transactions and orders with regard to the involved items, their quantities, prices, discounts, methods of payment, and the customers' demographics for analysis in terms of buying behavior, sales trend, inventory management, and marketing strategy.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
# Set visual display settings
sns.set(style='whitegrid')

In [3]:
original_df = pd.read_csv('online_retail_sales_dataset.csv')

In [4]:
# View first 5 rows
original_df.head()

,transaction_id,timestamp,customer_id,product_id,product_category,quantity,price,discount,payment_method,customer_age,customer_gender,customer_location,total_amount
0,1,2023-01-01 00:00:00,1993,915,Home & Kitchen,8,103.30,0.23,Gift Card,27,Female,North America,636.33
1,2,2023-01-01 00:01:00,3474,553,Clothing,9,180.28,0.31,Gift Card,53,Other,South America,1119.54
2,3,2023-01-01 00:02:00,4564,248,Beauty & Personal Care,7,81.58,0.27,Debit Card,34,Other,North America,416.87
3,4,2023-01-01 00:03:00,1133,948,Clothing,3,235.20,0.00,Debit Card,50,Other,Australia,705.60
4,5,2023-01-01 00:04:00,3626,284,Books,9,453.00,0.34,Credit Card,23,Female,Australia,2690.82


In [5]:
# Vars

# specify numeric columns
numeric_columns = original_df.select_dtypes(include=['int64', 'float64']).columns

# specify columns to eliminate
columns_to_exclude = ['transaction_id', 'customer_id', 'product_id']

# columns for usual numeric process
cols_to_process = ['quantity', 'price', 'discount', 'customer_age', 'total_amount']

# filter out the specified numeric columns
filtered_numeric_columns = [col for col in numeric_columns if col not in columns_to_exclude]

# DF of numeric columns to process
filtered_numeric_df = original_df[filtered_numeric_columns]

# Create a copy of the original DataFrame
original_df_copy = original_df.copy()

In [6]:
def convert_column_types(df, column_types):
    for column, data_type in column_types.items():
        if data_type == 'int64':
            df[column] = pd.to_numeric(df[column], errors='coerce').fillna(0).astype(np.int64)
        elif data_type == 'object':
            df[column] = df[column].astype(str)
        elif data_type == 'float64':
            df[column] = pd.to_numeric(df[column], errors='coerce').fillna(0.0).astype(np.float64)
        elif data_type == 'datetime64[ns]':
            df[column] = pd.to_datetime(df[column], errors='coerce')
        else:
            raise ValueError(f"Unsupported data type: {data_type}")
    return df

def confirm_column_types(df, column_types):
    for column, data_type in column_types.items():
        if df[column].dtype != data_type:
            return False
    return True

column_types = {
   'transaction_id': 'int64', 'timestamp': 'object', 'customer_id': 'int64', 'product_id': 'int64',
       'product_category': 'object', 'quantity': 'int64', 'price': 'float64', 'discount': 'float64', 'payment_method': 'object',
       'customer_age': 'int64', 'customer_gender': 'object', 'customer_location': 'object', 'total_amount': 'float64'
}

# Apply the function to new DF
### First conversion ###
dataType_converted_df = convert_column_types(original_df_copy, column_types)
print(dataType_converted_df.dtypes)

transaction_id         int64
timestamp             object
customer_id            int64
product_id             int64
product_category      object
quantity               int64
price                float64
discount             float64
payment_method        object
customer_age           int64
customer_gender       object
customer_location     object
total_amount         float64
dtype: object


In [7]:
# Columns included
original_df.columns

Index(['transaction_id', 'timestamp', 'customer_id', 'product_id',
       'product_category', 'quantity', 'price', 'discount', 'payment_method',
       'customer_age', 'customer_gender', 'customer_location', 'total_amount'],
      dtype='object')

In [8]:
# Shape of DF (rows, columns)
original_df.shape

(1000000, 13)

In [9]:
# Convert to timestamp column to datetime object
original_df['timestamp'] = pd.to_datetime(original_df['timestamp']).dt.date

# Format the datetime object
# original_df['timestamp'] = original_df['timestamp'].dt.strftime('%B %d, %Y')

In [10]:
original_df.head()

,transaction_id,timestamp,customer_id,product_id,product_category,quantity,price,discount,payment_method,customer_age,customer_gender,customer_location,total_amount
0,1,2023-01-01,1993,915,Home & Kitchen,8,103.30,0.23,Gift Card,27,Female,North America,636.33
1,2,2023-01-01,3474,553,Clothing,9,180.28,0.31,Gift Card,53,Other,South America,1119.54
2,3,2023-01-01,4564,248,Beauty & Personal Care,7,81.58,0.27,Debit Card,34,Other,North America,416.87
3,4,2023-01-01,1133,948,Clothing,3,235.20,0.00,Debit Card,50,Other,Australia,705.60
4,5,2023-01-01,3626,284,Books,9,453.00,0.34,Credit Card,23,Female,Australia,2690.82


## Outliers Elimination

In [11]:
def count_outliers_iqr(dataframe, columns):
    outlier_counts = {}
    for column in columns:
        # Calculate Q1 (25th percentile) and Q3 (75th percentile)
        Q1 = dataframe[column].quantile(0.25)
        Q3 = dataframe[column].quantile(0.75)
        IQR = Q3 - Q1
        
        # Define the outlier boundaries
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Count the outliers
        outliers = dataframe[(dataframe[column] < lower_bound) | (dataframe[column] > upper_bound)]
        outlier_counts[column] = outliers.shape[0]  # Number of outliers
    
    return outlier_counts

# Count outliers in the specified numeric columns
outlier_counts = count_outliers_iqr(original_df, cols_to_process)

print(f"Number of outliers in total_amount column: {outlier_counts}")

Number of outliers in total_amount column: {'quantity': 0, 'price': 0, 'discount': 0, 'customer_age': 0, 'total_amount': 21985}


In [12]:
def remove_outliers_iqr(df, numeric_columns):
    df_filtered = df.copy()

    # For each numeric column, Q1 and Q3 are calculated using quantile(), and then IQR is computed.
    
    for column in numeric_columns:
        Q1 = df_filtered[column].quantile(0.25)  # First quartile (25th percentile)
        Q3 = df_filtered[column].quantile(0.75)  # Third quartile (75th percentile)
        IQR = Q3 - Q1  # Interquartile range
        
        # Define bounds for outliers
        # The lower and upper bounds for identifying outliers are established.
        lower_bound = Q1 - 0.5 * IQR
        upper_bound = Q3 + 0.5 * IQR
        
        # Remove outliers
        # Rows containing outliers are removed based on these bounds
        df_filtered = df_filtered[(df_filtered[column] >= lower_bound) & (df_filtered[column] <= upper_bound)]

    # Cleaned DataFrame without outliers is returned
    return df_filtered


original_df = remove_outliers_iqr(original_df, cols_to_process)

# Caveat: must run this function several times to remove the outlier completely

outlier_counts = count_outliers_iqr(original_df, cols_to_process)

In [13]:
print(f"Number of outliers in total_amount column after deletion: {outlier_counts}")

Number of outliers in total_amount column after deletion: {'quantity': 0, 'price': 0, 'discount': 0, 'customer_age': 0, 'total_amount': 0}


In [14]:
original_df.head()

,transaction_id,timestamp,customer_id,product_id,product_category,quantity,price,discount,payment_method,customer_age,customer_gender,customer_location,total_amount
0,1,2023-01-01,1993,915,Home & Kitchen,8,103.30,0.23,Gift Card,27,Female,North America,636.33
1,2,2023-01-01,3474,553,Clothing,9,180.28,0.31,Gift Card,53,Other,South America,1119.54
2,3,2023-01-01,4564,248,Beauty & Personal Care,7,81.58,0.27,Debit Card,34,Other,North America,416.87
5,6,2023-01-01,3349,802,Electronics,2,48.95,0.16,Gift Card,28,Male,South America,82.24
6,7,2023-01-01,1136,971,Clothing,6,68.70,0.45,PayPal,58,Male,North America,226.71


In [15]:
# Save the cleaned version to another csv
original_df.to_csv('cleaned_online_retail_sales_dataset.csv', index=False, encoding='utf-8')

In [16]:
# Verify the saved file

df = pd.read_csv('cleaned_online_retail_sales_dataset.csv')

df.head()

,transaction_id,timestamp,customer_id,product_id,product_category,quantity,price,discount,payment_method,customer_age,customer_gender,customer_location,total_amount
0,1,2023-01-01,1993,915,Home & Kitchen,8,103.30,0.23,Gift Card,27,Female,North America,636.33
1,2,2023-01-01,3474,553,Clothing,9,180.28,0.31,Gift Card,53,Other,South America,1119.54
2,3,2023-01-01,4564,248,Beauty & Personal Care,7,81.58,0.27,Debit Card,34,Other,North America,416.87
3,6,2023-01-01,3349,802,Electronics,2,48.95,0.16,Gift Card,28,Male,South America,82.24
4,7,2023-01-01,1136,971,Clothing,6,68.70,0.45,PayPal,58,Male,North America,226.71


In [18]:
df.shape

(827986, 13)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 827986 entries, 0 to 827985
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   transaction_id     827986 non-null  int64  
 1   timestamp          827986 non-null  object 
 2   customer_id        827986 non-null  int64  
 3   product_id         827986 non-null  int64  
 4   product_category   827986 non-null  object 
 5   quantity           827986 non-null  int64  
 6   price              827986 non-null  float64
 7   discount           827986 non-null  float64
 8   payment_method     827986 non-null  object 
 9   customer_age       827986 non-null  int64  
 10  customer_gender    827986 non-null  object 
 11  customer_location  827986 non-null  object 
 12  total_amount       827986 non-null  float64
dtypes: float64(3), int64(5), object(5)
memory usage: 82.1+ MB


In [20]:
outlier_counts = count_outliers_iqr(df, cols_to_process)

In [21]:
print(f"Number of outliers in total_amount column: {outlier_counts}")

Number of outliers in total_amount column: {'quantity': 0, 'price': 0, 'discount': 0, 'customer_age': 0, 'total_amount': 0}
